In [ ]:
!pip install soda-core soda-core-contracts python-dotenv pyyaml soda-core-pandas-dask numpy==1.26.4

In [67]:
# !pip install soda-core soda-core-contracts python-dotenv pyyaml soda-core-pandas-dask numpy==1.26.4

import json
import dask.dataframe as dd
from soda.scan import Scan

scan = Scan()
scan.set_scan_definition_name("test")
scan.set_data_source_name("dask")

ddf = dd.read_csv('data/emoji_usage_dataset.csv')

scan.add_dask_dataframe(dataset_name="emoji", dask_df=ddf)

# Checks para usar: https://docs.soda.io/soda-cl/soda-cl-overview.html
checks = """
checks for emoji:
    - row_count > 0
    - invalid_count("User Age") = 0:
        valid min: 26
        valid max: 52
    - avg("User Age") > 37
    - duplicate_count(Emoji) = 0
# Checks for schema changes
    - schema:
        name: Find forbidden, missing, or wrong type
        warn:
            when required column missing: [Emoji, "User Gender"]
            when forbidden column present: [credit_card]
            when wrong column type:
                Platform: varchar
        fail:
            when forbidden column present: [pii*]
            when wrong column index:
                "User Age": 1
"""

scan.add_sodacl_yaml_str(checks)
scan.set_verbose(True)
scan.execute()
resultados_json = scan.get_scan_results()

json_path = "output/resultados.json"
with open(json_path, "w") as outfile:
    outfile.write(json.dumps(resultados_json, indent = 4))

INFO:soda.scan:[16:12:41] Soda Core 3.3.13
INFO:soda.scan:[16:12:41] Using DefaultSampler
ERROR:soda.scan:[16:12:41] Query execution error in 19.dask.emoji.Emoji.duplicate_count: SchemaError(FieldNotFound { field: Column { relation: None, name: "emoji" }, valid_fields: [Column { relation: Some(Bare { table: "emoji" }), name: "Emoji" }, Column { relation: Some(Bare { table: "emoji" }), name: "Context" }, Column { relation: Some(Bare { table: "emoji" }), name: "Platform" }, Column { relation: Some(Bare { table: "emoji" }), name: "User Age" }, Column { relation: Some(Bare { table: "emoji" }), name: "User Gender" }] })

WITH frequencies AS (
    SELECT COUNT(*) AS frequency
    FROM emoji
    WHERE Emoji IS NOT NULL
    GROUP BY Emoji)
SELECT COUNT(*)
FROM frequencies
WHERE frequency > 1
ERROR:soda.scan:  | SchemaError(FieldNotFound { field: Column { relation: None, name: "emoji" }, valid_fields: [Column { relation: Some(Bare { table: "emoji" }), name: "Emoji" }, Column { relation: Some(Ba